In [3]:
# imports
import pymysql
#pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
from sqlalchemy.types import *
from sqlalchemy_utils import database_exists
import pandas as pd
import numpy as np


In [4]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
#print(basics)

In [5]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [6]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

## Cleaning

In [7]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [8]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1993
1,tt0000002,5.8,267
2,tt0000003,6.5,1874
3,tt0000004,5.5,177
4,tt0000005,6.2,2658


In [9]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


# Akas
     replacing \N with nan
     filtering US only movies

In [10]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37144830 entries, 0 to 37144829
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


In [11]:
akas.replace({'\\N':np.nan}, inplace=True)

In [12]:
#akas = akas.loc[akas["region"] != 'US']
#akas.drop(akas.loc[akas["region"] != 'US'], inplace=True)
akas.drop(akas[akas['region'] != 'US'].index, inplace = True)

In [37]:
# check values in 'region'
akas['region'].value_counts()

US    1465096
Name: region, dtype: int64

In [13]:
print(akas)

            titleId  ordering                                      title  \
5         tt0000001         6                                 Carmencita   
14        tt0000002         7                     The Clown and His Dogs   
33        tt0000005        10                           Blacksmith Scene   
36        tt0000005         1                        Blacksmithing Scene   
41        tt0000005         6                        Blacksmith Scene #1   
...             ...       ...                                        ...   
37144356  tt9916560         1  March of Dimes Presents: Once Upon a Dime   
37144426  tt9916620         1                          The Copeland Case   
37144515  tt9916702         1              Loving London: The Playground   
37144558  tt9916756         1                   Pretty Pretty Black Girl   
37144574  tt9916764         1                                         38   

         region language        types             attributes isOriginalTitle  
5       

# Basics

In [14]:
keepers =basics['tconst'].isin(akas['titleId'])
keepers

0            True
1            True
2           False
3           False
4            True
            ...  
10150065    False
10150066    False
10150067    False
10150068    False
10150069    False
Name: tconst, Length: 10150070, dtype: bool

In [15]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
...,...,...,...,...,...,...,...,...,...
10149931,tt9916560,tvMovie,March of Dimes Presents: Once Upon a Dime,March of Dimes Presents: Once Upon a Dime,0,1963,\N,58,Family
10149960,tt9916620,movie,The Copeland Case,The Copeland Case,0,\N,\N,\N,Drama
10149998,tt9916702,short,Loving London: The Playground,Loving London: The Playground,0,\N,\N,\N,"Drama,Short"
10150021,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,0,2019,\N,\N,Short


In [16]:
basics.replace({'\\N':np.nan}, inplace=True)

In [17]:

basics.drop(basics[basics['runtimeMinutes'].isnull()].index, inplace = True)

In [18]:
#import math
print(basics)
#print(math.NaN)

             tconst titleType                                 primaryTitle  \
0         tt0000001     short                                   Carmencita   
1         tt0000002     short                       Le clown et ses chiens   
4         tt0000005     short                             Blacksmith Scene   
5         tt0000006     short                            Chinese Opium Den   
6         tt0000007     short  Corbett and Courtney Before the Kinetograph   
...             ...       ...                                          ...   
10149763  tt9916214     short                              Drown the Clown   
10149783  tt9916254     video                         Big Tit Cream Pie 32   
10149829  tt9916348     video                        Ancient World Exposed   
10149836  tt9916362     movie                                        Coven   
10149931  tt9916560   tvMovie    March of Dimes Presents: Once Upon a Dime   

                                        originalTitle isAdult s

In [19]:
basics.drop(basics[basics['genres'].isnull()].index, inplace = True)

In [20]:
print(basics)

             tconst titleType                                 primaryTitle  \
0         tt0000001     short                                   Carmencita   
1         tt0000002     short                       Le clown et ses chiens   
4         tt0000005     short                             Blacksmith Scene   
5         tt0000006     short                            Chinese Opium Den   
6         tt0000007     short  Corbett and Courtney Before the Kinetograph   
...             ...       ...                                          ...   
10149763  tt9916214     short                              Drown the Clown   
10149783  tt9916254     video                         Big Tit Cream Pie 32   
10149829  tt9916348     video                        Ancient World Exposed   
10149836  tt9916362     movie                                        Coven   
10149931  tt9916560   tvMovie    March of Dimes Presents: Once Upon a Dime   

                                        originalTitle isAdult s

In [21]:
basics.drop(basics[basics['titleType'] != 'movie'].index, inplace = True)

In [22]:
print(basics)

             tconst titleType                    primaryTitle  \
8         tt0000009     movie                      Miss Jerry   
144       tt0000147     movie   The Corbett-Fitzsimmons Fight   
570       tt0000574     movie     The Story of the Kelly Gang   
587       tt0000591     movie                The Prodigal Son   
672       tt0000679     movie  The Fairylogue and Radio-Plays   
...             ...       ...                             ...   
10149425  tt9915436     movie               Vida em Movimento   
10149603  tt9915872     movie            The Last White Witch   
10149743  tt9916170     movie                   The Rehearsal   
10149752  tt9916190     movie                       Safeguard   
10149836  tt9916362     movie                           Coven   

                           originalTitle isAdult startYear endYear  \
8                             Miss Jerry       0      1894     NaN   
144        The Corbett-Fitzsimmons Fight       0      1897     NaN   
570      

In [23]:
basics['startYear'].value_counts

<bound method IndexOpsMixin.value_counts of 8           1894
144         1897
570         1906
587         1907
672         1908
            ... 
10149425    2019
10149603    2019
10149743    2019
10149752    2020
10149836    2020
Name: startYear, Length: 205133, dtype: object>

In [24]:
basics.isna().sum()
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205133 entries, 8 to 10149836
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          205133 non-null  object
 1   titleType       205133 non-null  object
 2   primaryTitle    205133 non-null  object
 3   originalTitle   205133 non-null  object
 4   isAdult         205133 non-null  object
 5   startYear       201529 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  205133 non-null  object
 8   genres          205133 non-null  object
dtypes: object(9)
memory usage: 15.7+ MB


In [26]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205133 entries, 8 to 10149836
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          205133 non-null  object 
 1   titleType       205133 non-null  object 
 2   primaryTitle    205133 non-null  object 
 3   originalTitle   205133 non-null  object 
 4   isAdult         205133 non-null  object 
 5   startYear       201529 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  205133 non-null  object 
 8   genres          205133 non-null  object 
dtypes: float64(1), object(8)
memory usage: 15.7+ MB


In [51]:
basics['startYear'] = basics['startYear'].astype('int')

In [52]:
# keep only 'startYear' from 2000 to 2022, including 2000 and 2022
basics = basics[(basics['startYear'] >= 2000) &
                (basics['startYear'] <= 2022)]

# check
basics['startYear'].value_counts().sort_index(ascending = False)

2022    5529
2021    5618
2020    5478
2019    6458
2018    6326
2017    6320
2016    6063
2015    6150
2014    6380
2013    6109
2012    5684
2011    5227
2010    4785
2009    4096
2008    3241
2007    2815
2006    2650
2005    2358
2004    2043
2003    1822
2002    1686
2001    1666
2000    1558
Name: startYear, dtype: int64

In [53]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
80549,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,NaN,94,Horror
...,...,...,...,...,...,...,...,...,...
10149209,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
10149603,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019,NaN,97,"Comedy,Drama,Fantasy"
10149743,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
10149752,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [54]:
basics.drop(basics[basics['genres'] == 'Documentary'].index, inplace = True)

In [55]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
80549,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,NaN,94,Horror
...,...,...,...,...,...,...,...,...,...
10149209,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
10149603,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019,NaN,97,"Comedy,Drama,Fantasy"
10149743,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
10149752,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


# Rankings

In [56]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1993
1,tt0000002,5.8,267
2,tt0000003,6.5,1874
3,tt0000004,5.5,177
4,tt0000005,6.2,2658


In [57]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348135 entries, 0 to 1348134
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1348135 non-null  object 
 1   averageRating  1348135 non-null  float64
 2   numVotes       1348135 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.9+ MB


In [58]:
us_movies = basics['tconst'].isin(akas['titleId'])
us_movies
#new_ratings

34800       True
61112       True
67486       True
67664       True
80549       True
            ... 
10149209    True
10149603    True
10149743    True
10149752    True
10149836    True
Name: tconst, Length: 100062, dtype: bool

In [59]:
basics_new = basics[us_movies]
basics_new

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
80549,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,NaN,94,Horror
...,...,...,...,...,...,...,...,...,...
10149209,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
10149603,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019,NaN,97,"Comedy,Drama,Fantasy"
10149743,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
10149752,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [60]:
# create filter for ratings to only include the US by using akas
# (which has already been cut down to only include US)
us_movies = ratings['tconst'].isin(akas['titleId'])

# check
us_movies

0           True
1           True
2          False
3          False
4           True
           ...  
1348130    False
1348131    False
1348132    False
1348133    False
1348134    False
Name: tconst, Length: 1348135, dtype: bool

## Review

In [61]:
basics_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100062 entries, 34800 to 10149836
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          100062 non-null  object
 1   titleType       100062 non-null  object
 2   primaryTitle    100062 non-null  object
 3   originalTitle   100062 non-null  object
 4   isAdult         100062 non-null  object
 5   startYear       100062 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  100062 non-null  object
 8   genres          100062 non-null  object
dtypes: int32(1), object(8)
memory usage: 7.3+ MB


In [62]:
# check info for akas
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465096 entries, 0 to 1465095
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1465096 non-null  object 
 1   ordering         1465096 non-null  int64  
 2   title            1465096 non-null  object 
 3   region           1465096 non-null  object 
 4   language         4130 non-null     object 
 5   types            983650 non-null   object 
 6   attributes       47469 non-null    object 
 7   isOriginalTitle  1463754 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 89.4+ MB


In [63]:
# check info for ratings
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348135 entries, 0 to 1348134
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1348135 non-null  object 
 1   averageRating  1348135 non-null  float64
 2   numVotes       1348135 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.9+ MB


## Save

In [64]:
# make data folder
import os
os.makedirs('Data/', exist_ok = True)

# check
os.listdir('Data/')

['final_tmdb_data_by_year',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [65]:
# save each file to a compressed csv file in data folder
basics_new.to_csv('Data/title_basics.csv.gz', 
              compression = 'gzip', 
              index = False)

# check
basics_new = pd.read_csv('Data/title_basics.csv.gz',
                    low_memory = False)

basics_new.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,NaN,94,Horror


In [66]:
# save each file to a compressed csv file in data folder
akas.to_csv('Data/title_akas.csv.gz', 
              compression = 'gzip', 
              index = False)

# check
akas = pd.read_csv('Data/title_akas.csv.gz',
                    low_memory = False)

akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [67]:
# save each file to a compressed csv file in data folder
ratings.to_csv('Data/title_ratings.csv.gz', 
              compression = 'gzip', 
              index = False)

# check
ratings = pd.read_csv('Data/title_ratings.csv.gz',
                    low_memory = False)

ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1993
1,tt0000002,5.8,267
2,tt0000003,6.5,1874
3,tt0000004,5.5,177
4,tt0000005,6.2,2658
